In [2]:
import os
import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Activation, Dropout, BatchNormalization

!pip install wandb
import wandb
from wandb.keras import WandbCallback
!pip install wandb --upgrade

In [3]:
no_of_filters = [8, 16, 16, 32, 32]
kernel_size =   [3 , 5 , 5 , 5 , 5]
activation  =   tf.nn.relu
dense_layer = 100
image_size = 200
batch_size = 50
epochs = 10

#Dropout and Batch Normalization
dropout=0.0
batch_norm=False

augment_data=False

In [4]:
#TensorFlow - Building the Model
def createCNN(image_size, no_of_filters, kernel_size, activation, batch_norm, dropout, dense_size):

    modeltf = Sequential()

    # layer1
    modeltf.add(Conv2D(input_shape=(image_size, image_size, 3), filters=no_of_filters[0], kernel_size=kernel_size[0], strides=1, padding="same", activation=activation))
    if batch_norm:
        modeltf.add(BatchNormalization())
    modeltf.add(AveragePooling2D(pool_size=2, strides=2))

    # layer2
    modeltf.add(Conv2D(no_of_filters[1], kernel_size=kernel_size[1], strides=1, padding="same", activation=activation))
    if batch_norm:
        modeltf.add(BatchNormalization())
    modeltf.add(AveragePooling2D(pool_size=2, strides=2))

    # layer3
    modeltf.add(Conv2D(no_of_filters[2], kernel_size=kernel_size[2], strides=1, padding="same", activation=activation))
    if batch_norm:
        modeltf.add(BatchNormalization())
    modeltf.add(AveragePooling2D(pool_size=2, strides=2))

    # layer4
    modeltf.add(Conv2D(no_of_filters[3], kernel_size=kernel_size[3], strides=1, padding="same", activation=activation))
    if batch_norm:
        modeltf.add(BatchNormalization())
    modeltf.add(AveragePooling2D(pool_size=2, strides=2))

    # layer5
    modeltf.add(Conv2D(no_of_filters[4], kernel_size=kernel_size[4], strides=1, padding="same", activation=activation))
    if batch_norm:
        modeltf.add(BatchNormalization())
    modeltf.add(AveragePooling2D(pool_size=2, strides=2))

    modeltf.add(Flatten())

    # dense layer
    modeltf.add(Dense(dense_layer, activation=activation))
    modeltf.add(Dropout(dropout))

    # output layer
    modeltf.add(Dense(10, activation=tf.nn.softmax))
    
    return modeltf

In [5]:
# Prepare the dataset for training and testing
def prepare_dataset(DATA_DIR="inaturalist_12K", augment_data=False):
    train_dir = os.path.join(DATA_DIR, "train")
    test_dir = os.path.join(DATA_DIR, "val")

    if augment_data:
        train_datagen = ImageDataGenerator(rescale=1./255,
                                          rotation_range=90,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True)
        test_datagen = ImageDataGenerator(rescale=1./255)

    else:
        train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', subset="training")
    val_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', subset="validation")
    test_generator = test_datagen.flow_from_directory(test_dir, target_size=(image_size, image_size), batch_size=batch_size)
    
    return train_generator, val_generator, test_generator

In [7]:
#Customise run names for WandB to enhance readability
def setRunName(no_of_filters = [8, 16, 16, 32, 32], augment_data=False, dropout=0.0, batch_norm=False):
    
    augment_data_options = {True: "Y", False: "N"}
    batch_norm_options = {True: "Y", False: "N"}

    run_name = "_".join(["filters", str(no_of_filters), "aug", augment_data_options[augment_data],
                      "drop", str(dropout), "norm", batch_norm_options[batch_norm]])
    
    return run_name;

In [8]:
#Testing
def test():

    config_defaults = {
        "no_of_filters_1": 8,
        "no_of_filters_2": 16,
        "no_of_filters_3": 16,
        "no_of_filters_4": 32,
        "no_of_filters_5": 32,
        "augment_data": True,
        "dropout": 0.0,
        "batch_norm": True,
        "epochs": 45,
        "dense_size": 128,
        "lr": 0.001
    }
    
    os.environ['WANDB_NOTEBOOK_NAME'] = 'A2_P1_2_best2.ipynb'

    wandb.init(config=config_defaults, magic=True)
    config = wandb.config
    wandb.run.name = setRunName([config.no_of_filters_1, config.no_of_filters_2, config.no_of_filters_3, config.no_of_filters_4, config.no_of_filters_5], config.augment_data, config.dropout, config.batch_norm)

    train_generator, val_generator, test_generator = prepare_dataset(augment_data=config.augment_data)
    model = createCNN(image_size, [config.no_of_filters_1, config.no_of_filters_2, config.no_of_filters_3, config.no_of_filters_4, config.no_of_filters_5], kernel_size, activation, config.batch_norm, config.dropout, config.dense_size)
    model.compile(optimizer=keras.optimizers.Adam(config.lr), loss="categorical_crossentropy", metrics="accuracy")
    model.fit(train_generator, epochs=config.epochs, validation_data=val_generator, callbacks=[WandbCallback()])

    print("Testing Model:")
    
    loss, acc = model.evaluate(test_generator, batch_size=batch_size)
    print(f'Model accuracy : {acc} and loss : {loss}')
    #predictions = model(test_generator[0][0])
    model.save("Best_model2.h5")
    

In [9]:
#Set up a sweep config
sweep_config = {
    "description": "Training, Checking the performance of CNN on validation data and Testing",
    "metric": {
    'name': 'val_accuracy',
    'goal': 'maximize'   
    },
    "method": "grid",
    "project": "DL_CS6910_Assignment2",
    "parameters": {
        "no_of_filters_1": {
            "values": [8]
        },
        "no_of_filters_2": {
            "values": [16]
        },
        "no_of_filters_3": {
            "values": [16]
        },
        "no_of_filters_4": {
            "values": [32]
        },
        "no_of_filters_5": {
            "values": [32]
        },
        "augment_data": {
            "values": [True]
        },
        "dropout": {
            "values": [0.0]
        },
        "batch_norm": {
            "values": [True]
        },
        "epochs": {
            "values": [45]
        },
        "dense_size": {
            "values": [128]
        },
        "lr": {
            "values": [0.001]
        }
    }
}

# creating the sweep
sweep_id = wandb.sweep(sweep_config, project="DL_CS6910_Assignment2")

Create sweep with ID: 4z3r2ipd
Sweep URL: https://wandb.ai/cs21m029_keyur_raval/DL_CS6910_Assignment2/sweeps/4z3r2ipd


In [10]:
wandb.agent(sweep_id, function=test)

wandb: Agent Starting Run: 0i4bmues with config:
wandb: 	augment_data: True
wandb: 	batch_norm: True
wandb: 	dense_size: 128
wandb: 	dropout: 0
wandb: 	epochs: 45
wandb: 	lr: 0.001
wandb: 	no_of_filters_1: 8
wandb: 	no_of_filters_2: 16
wandb: 	no_of_filters_3: 16
wandb: 	no_of_filters_4: 32
wandb: 	no_of_filters_5: 32
wandb: Currently logged in as: cs21m029_keyur_raval (use `wandb login --relogin` to force relogin)


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/45
180/180 [==============================] - 116s 620ms/step - loss: 2.2879 - accuracy: 0.2060 - val_loss: 2.5159 - val_accuracy: 0.0971
Epoch 2/45
180/180 [==============================] - 111s 617ms/step - loss: 2.0790 - accuracy: 0.2599 - val_loss: 2.2868 - val_accuracy: 0.1782
Epoch 3/45
180/180 [==============================] - 111s 616ms/step - loss: 2.0055 - accuracy: 0.2849 - val_loss: 2.1917 - val_accuracy: 0.2563
Epoch 4/45
180/180 [==============================] - 111s 615ms/step - loss: 1.9714 - accuracy: 0.2979 - val_loss: 2.0558 - val_accuracy: 0.2763
Epoch 5/45
180/180 [==============================] - 111s 616ms/step - loss: 1.9326 - accuracy: 0.3126 - val_loss: 1.9644 - val_accuracy: 0.3033
Epoch 6/45
180/180 [==============================] - 111s 616ms/step - loss: 1.9053 - accuracy: 0.3232 - val_loss: 1.9211 - val_accuracy: 0.3

accuracy,▁▂▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇███████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▆▆▆▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_accuracy,▁▃▅▅▆▆▆▆▆▇▆▆▆▆▇▇▇▇▇█▇▆▇▆▇▇████▇█████▇██▇
val_loss,█▆▅▄▃▂▃▃▃▂▃▃▃▂▂▁▂▁▂▂▂▃▂▃▁▁▁▂▁▁▂▁▁▁▁▂▃▂▁▁
accuracy,0.51678
best_epoch,26
best_val_loss,1.77281
epoch,44
loss,1.38242
val_accuracy,0.37538


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
